In [6]:
import cv2
import pandas as pd
import time
import matplotlib.pylab as plt
import numpy as np
import math
from PIL import Image
from deepface.extendedmodels import Age, Gender, Race, Emotion
from deepface import DeepFace
from pathlib import Path
from deepface.commons import distance
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.preprocessing import image

#from deepface import DeepFace backends = ['opencv', 'ssd', 'dlib', 'mtcnn']

detector = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")
emotion_model = Emotion.loadModel()


def alignment_procedure(img, left_eye, right_eye):  # find degree and rotate image
	#this function aligns given face in img based on left and right eye coordinates
	left_eye_x, left_eye_y = left_eye
	right_eye_x, right_eye_y = right_eye
	
	#-----------------------
	#find rotation direction
		
	if left_eye_y > right_eye_y:
		point_3rd = (right_eye_x, left_eye_y)
		direction = -1 #rotate same direction to clock
	else:
		point_3rd = (left_eye_x, right_eye_y)
		direction = 1 #rotate inverse direction of clock
	
	#-----------------------
	#find length of triangle edges
	
	a = distance.findEuclideanDistance(np.array(left_eye), np.array(point_3rd))
	b = distance.findEuclideanDistance(np.array(right_eye), np.array(point_3rd))
	c = distance.findEuclideanDistance(np.array(right_eye), np.array(left_eye))
	
	#-----------------------
	
	#apply cosine rule
			
	if b != 0 and c != 0: #this multiplication causes division by zero in cos_a calculation
		
		cos_a = (b*b + c*c - a*a)/(2*b*c)
		angle = np.arccos(cos_a) #angle in radian
		angle = (angle * 180) / math.pi #radian to degree
		
		#-----------------------
		#rotate base image
		
		if direction == -1:
			angle = 90 - angle
		
		img = Image.fromarray(img)
		img = np.array(img.rotate(direction * angle))
	
	#-----------------------
	
	return img #return img anyway


def align_face(img):  # rotate face to horizontal
	
	home = str(Path.home())	
	
	from mtcnn import MTCNN
	mtcnn_detector = MTCNN()
	detections = mtcnn_detector.detect_faces(img)
		
	if len(detections) > 0:
		detection = detections[0]
		
		keypoints = detection["keypoints"]
		left_eye = keypoints["left_eye"]
		right_eye = keypoints["right_eye"]
		
		img = alignment_procedure(img, left_eye, right_eye)
		
	return img #return img anyway


def get_tag_emotion(image_path, tx, ty):  # return tag list
    image_file = cv2.imread(image_path)  # read image
    if image_file is None:
        return
    base_img = image_file.copy()  # back up original image
    original_size = base_img.shape

    target_size = (ty, tx)  # The higher the number, the easier it is to find a small face.
    image_file = cv2.resize(image_file, target_size)
    aspect_ratio_x = (original_size[1] / target_size[1])
    aspect_ratio_y = (original_size[0] / target_size[0])

    # detector expects (1, 3, 300, 300) shaped input
    imageBlob = cv2.dnn.blobFromImage(image=image_file)
    # imageBlob = np.expand_dims(np.rollaxis(image, 2, 0), axis = 0)
    detector.setInput(imageBlob)
    detections = detector.forward()

    column_labels = ["img_id", "is_face", "confidence", "left", "top", "right", "bottom"]
    detections_df = pd.DataFrame(detections[0][0], columns=column_labels)

    # 0: background, 1: face, confidence 가 높은게 얼굴일 확률이 높다
    detections_df = detections_df[detections_df['is_face'] == 1]
    detections_df = detections_df[detections_df['confidence'] > 0.90]
    
    detections_df['left'] = (detections_df['left'] * tx).astype(int)  # 좌표값을 정수형 데이터로 변환
    detections_df['bottom'] = (detections_df['bottom'] * ty).astype(int)
    detections_df['right'] = (detections_df['right'] * tx).astype(int)
    detections_df['top'] = (detections_df['top'] * ty).astype(int)
    
    detections_df = detections_df[detections_df['right'] < original_size[1]]
    detections_df = detections_df[detections_df['bottom'] < original_size[0]]
    
    face_list_df = pd.DataFrame(columns=['left', 'top', 'right', 'bottom'])  # pos of faces

    for i, instance in detections_df.iterrows():  # 
        confidence_score = str(round(100*instance["confidence"], 2))+" %"
        left = instance["left"]
        right = instance["right"]
        bottom = instance["bottom"]
        top = instance["top"]

        face_list_df = face_list_df.append({'left': left, 'top': top, 'right': right, 'bottom': bottom}, ignore_index=True)
        # cv2.putText(base_img, confidence_score, (int(left*aspect_ratio_x), int(top*aspect_ratio_y-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    face_list_df = face_list_df.astype(int)
    
    emotion_res_list = list()
    
    for i, instance in face_list_df.iterrows():  # each face
        detected_face = base_img[int(instance['top'] * aspect_ratio_y):int(instance['bottom'] * aspect_ratio_y),
                        int(instance['left'] * aspect_ratio_x):int(instance['right'] * aspect_ratio_x)]  # get face
        #detected_face = detected_face[:, :, ::-1]
        #plt.imshow(detected_face)
        #plt.show()
        detected_face = align_face(detected_face)  # face rotate to horizontal        
        detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY)  # to grey
        face48 = cv2.resize(detected_face, (48,48))        
        img_pixels = image.img_to_array(face48)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255 #normalize input in [0, 1]
        
        emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
        emotion_predictions = emotion_model.predict(img_pixels)[0,:]  # get emotion info use emotion model
        sum_of_predictions = emotion_predictions.sum()
        
        #emotion_res = list()
        #emotion_res.append([emotion_labels[0], 100 * emotion_predictions[0] / sum_of_predictions])        
        emotion_res = [emotion_labels[0], 100 * emotion_predictions[0] / sum_of_predictions]
        
        for i in range(1, len(emotion_labels)):
            emotion_label = emotion_labels[i]
            emotion_prediction = 100 * emotion_predictions[i] / sum_of_predictions            
            if(emotion_res[1] < emotion_prediction):
                emotion_res = [emotion_labels[i], emotion_prediction]
        # print(emotion_res)
        emotion_res_list.append(emotion_res[0])

    for i, instance in face_list_df.iterrows(): # draw rectangle
        cv2.putText(base_img, emotion_res_list[i], (int(instance['left']*aspect_ratio_x), int(instance['top']*aspect_ratio_y-10)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
        cv2.rectangle(base_img, (int(instance['left'] * aspect_ratio_x), int(instance['top'] * aspect_ratio_y)),
                      (int(instance['right'] * aspect_ratio_x), int(instance['bottom'] * aspect_ratio_y)), (255, 255, 255), 3)
        
    imS = cv2.resize(base_img, (800, 800))
    cv2.imshow("test", imS)
    cv2.waitKey(5000)
    cv2.destroyAllWindows()
    return emotion_res_list

start = time.time()
imgpath = "images\\image17.jpg"
print(get_tag_emotion(imgpath, 750, 750))
#get_tag_emotion(imgpath, 500, 500)
#get_tag_emotion(imgpath, 850, 850)
#get_tag_emotion(imgpath, 1000, 700)
#get_tag_emotion(imgpath, 700, 1000)
print("time :", time.time() - start)

#result  = DeepFace.verify(imgpath, "images\\image8.jpg")
#print(result)
#demography = DeepFace.analyze(imgpath, actions=['emotion'])
#print("Emotion: ", demography["dominant_emotion"])


['happy', 'sad', 'neutral', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'fear', 'happy', 'neutral', 'neutral', 'neutral', 'happy', 'happy', 'happy', 'neutral']
time : 25.6879243850708


In [5]:
import sys


print(sys.version)

3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
